In [1]:
"""Global module calls"""

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

In [2]:
"""Classes"""

class FCC:
    """
    Creates an object (whatever the user defines 'self' as) that has tied to it the array of points that are in an FCC
    lattice as well as the vectors needed that create that lattice from a superposition of basis vectors.
    
    ---
    Parameters:
    a, float: The value of the lattice constant along the x axis of the crystal geometry.
    b, float: The value of the lattice constant along the y axis of the crystal geometry.
    c, float: The value of the lattice constant along the z axis of the crystal geometry.
    basis_vectors, list/array: A list/array of the points that can be used to construct the 'lattice_vectors.'
    lattice_points, list/array: A list/array of the points that make up the lattice.
    lattice_vectors, list/array: A list/array of the vectors that can be used to construct the lattice.
    """
    
    def __init__(self, a, b, c):
        """
        Initializes all properties of the FCC crystal lattice object tied to 'self.'
        
        # a, float: The value of the lattice constant along the x axis of the crystal geometry.
        # b, float: The value of the lattice constant along the y axis of the crystal geometry.
        # c, float: The value of the lattice constant along the z axis of the crystal geometry.
        
        ---
        Returns: Nothing since this function initializes the object and the properties of the object.
        
        """
        
        self.a = a
        self.b = b
        self.c = c
        self.cryst_basis = [[a/2, b/2, 0], [a/2, 0, c/2], [0, b/2, c/2]]
        self.lat_pnts = self.fcc()
        self.recip = self.bcc()

        
#     def cryst_vecs(self):
#         """
#         Uses a not-necessarily-orthogonal basis governed by the crystal lattice to recreate the lattice of points in
#         'create_vecs.'
        
#         ---
#         Returns: a/an list/array of the vectors needed to translate to every lattice point from one lattice point. These
#         lattice points can also be used to create the lattice if one starts at one of the bottom corners of the lattice.
#         Realized that this function is totally useless but I will keep it here just in case. It does have outdated
#         notation though.
#         """        
        
#         # Initializes the list/array.
#         lat_vec = []
        
#         # Literal hard coding of the points as vectors. Technically, the vectors I was trying to find are different but since the code cannot tell that, it does not matter.
#         for i in range(len(self.lattice_points)):
#             q, r, s = self.lattice_points[i][0], self.lattice_points[i][1], self.lattice_points[i][2]
#             l, m, n = ((r/self.b) - (s/self.c) + (q/self.a)), ((s/self.c) + (q/self.a) - (r/self.b)), ((r/self.b) + (s/self.c) - (q/self.a))
#             lat_vec.append((l * np.array(self.basis_vectors[0]) + m * np.array(self.basis_vectors[1]) + n * np.array(self.basis_vectors[2])))
            
#         return np.array(lat_vec)
    
    def fcc(self):
        """
        Creates a list of the points on the Face Centered Cubic crystal lattice.
        
        ---
        Parameters:
        self, object: The object itself.
        
        ---
        Returns: A list of vectors that point to each point of the crystal lattice using a standard Cartesian basis.
        """
        
        fcc_lattice_points = [[0, 0, 0], [self.a, 0, 0], [0, self.b, 0], [0, 0, self.c], [self.a, self.b, 0], [self.a, 0, self.c], [0, self.b, self.c], [self.a, self.b, self.c],
                         [self.a/2, self.b/2, 0], [self.a/2, 0, self.c/2], [0 , self.b/2, self.c/2], [self.a/2, self.b/2, self.c], [self.a/2, self.b, self.c/2], [self.a, self.b/2, self.c/2]]
        
        return fcc_lattice_points
    
    def bcc(self):
        """
        Creates a list of the points in the reciprocal lattice of the original FCC lattice. Thankfully, no math has to be
        done because this is simply a BCC lattice. I am most likely incorrect but these could serve as k values for the
        Brillouin Zone.
        
        ---
        Parameters:
        self, object: The object itself.
        
        ---
        Returns: A list of the vectors in the BCC lattice.
        """
        
        bcc_lattice_points = [[0, 0, 0], [self.a, 0, 0], [0, self.b, 0], [0, 0, self.c], [self.a, self.b, 0] ,[self.a, 0, self.c], [0, self.b, self.c], [self.a, self.b, self.c], [self.a/2, self.b/2, self.c/2]]
        
        return bcc_lattice_points

In [20]:
"""Functions"""

def atm_E(n, Z):
    """
    Creates the energy value of the Hydrogenic atomic wavefunctions.
    
    ---
    Parameters:
    n, int: The value of the quantum number n of the orbital.
    Z, int: The number of protons in nucleus of the atom.
    
    ---
    Returns: A scalar which represents the value of the energy of the atomic orbitals.
    """
    
    # Calling the energy term from the scipy module.
    from scipy.physics.hydrogen import E_nl
    
    return E_nl(n, Z)

def overlap(k, R_j, n, Z):

    """
    Creates an array of the values of the overlap integral multiplied by the summation term.
    
    ---
    Parameters:
    k, list/array: A (An) list (array) of the Brillouin Zone (k-space) values of interest.
    R_j, list/array: A (An) list (array) of the lattice points in the real space crystal lattice.
    n, int: The value of the quantum number n of the orbital.
    Z, int: The number of protons in nucleus of the atom.
    
    ---
    Returns: A (An) list (array) of the values for the overlap term to be plotted in k-space.
    """
    
    # Getting the wavefunctions from sympy.
    from sympy import Symbol
    from sympy.physics.hydrogen import Psi_nlm
    
    # Creating variables to be integrated over.
    r=Symbol("r", real=True, positive=True)
    phi=Symbol("phi", real=True)
    theta=Symbol("theta", real=True)
    
    # Creating the term(s)
    # if n=3, l=2,1,0, m=-2,-1,0,1,2
    
    #arrays filled with the quantum numbers made possible by 'n'.arrange l and m in arrays to access more easily
    l = np.arange(n)
    m = np.arange(1-n,n)
    mesh = np.array(np.meshgrid(l, m))
    q_num = mesh.T.reshape(-1, 2)
    for i in range(len(q_num)):
        a = q_num[i]
#        if np.greater(q_t[0],q_t[1]):
#            np.delete(q_num,i)
    
    overlap_integrand = []
    for i in range(len(q_num)):
        lm = q_num[i]
    #    overlap_integrand[i] = Psi_nlm(n,lm[0],lm[1],r,phi,theta,Z)

    overlap_term =[]
    
    return a

In [21]:
#testing
from sympy.physics.hydrogen import Psi_nlm
from sympy import Symbol
r=Symbol("r", real=True, positive=True)
X=Symbol("X", real=True, positive=True)
phi=Symbol("phi", real=True)
theta=Symbol("theta", real=True)

fcc = FCC(0.352, 0.352, 0.352)
R_j = fcc.lat_pnts
k_j =fcc.recip

o = overlap(k_j,R_j,3,28)
print(o)

[2 2]
